In [2]:
#Task 1: Imports and Utility - We're just doing some imports and enabling async to work within the Jupyter environment here, nothing too crazy!
#!pip install -qU numpy matplotlib plotly pandas scipy scikit-learn openai python-dotenv

In [3]:
from utils.text_utils import TextFileLoader, CharacterTextSplitter

In [4]:
from utils.vectordatabase import VectorDatabase
import asyncio

In [5]:
import nest_asyncio
nest_asyncio.apply()

In [6]:
#Task 2: Documents

# Loading Source Documents

# So, first things first, we need some documents to work with.

# While we could work directly with the .txt files (or whatever file-types you wanted to extend this to) we can instead do some batch processing of those documents at the beginning in order to store them in a more machine compatible format.

# In this case, we're going to parse our text file into a single document in memory.

# Let's look at the relevant bits of the TextFileLoader class:

# def load_file(self):
#         with open(self.path, "r", encoding=self.encoding) as f:
#             self.documents.append(f.read())

# We're simply loading the document using the built in open method, 
#and storing that output in our self.documents list.


In [7]:
text_loader = TextFileLoader("data/kingLear.txt")
documents = text_loader.load_documents()
len(documents)

1

In [8]:
print(documents[0][:100])

1606
THE TRAGEDY OF KING LEAR

by William Shakespeare




Dramatis Personae

      Lear, King of Bri


In [9]:
#Splitting Text Into Chunks

# As we can see, there is one massive document.

# We'll want to chunk the document into smaller parts so it's easier to pass the most relevant snippets to the LLM.

# There is no fixed way to split/chunk documents - and you'll need to rely on some intuition as well as knowing your data very well in order to build the most robust system.

# For this toy example, we'll just split blindly on length.

#     There's an opportunity to clear up some terminology here, for this course we will be stick to the following:

#         "source documents" : The .txt, .pdf, .html, ..., files that make up the files and information we start with in its raw format
#         "document(s)" : single (or more) text object(s)
#         "corpus" : the combination of all of our documents



In [10]:
text_splitter = CharacterTextSplitter()
split_documents = text_splitter.split_texts(documents)
len(split_documents)

216

In [11]:
#Let's take a look at some of the documents we've managed to split.

In [12]:
split_documents[0:1]

["1606\nTHE TRAGEDY OF KING LEAR\n\nby William Shakespeare\n\n\n\n\nDramatis Personae\n\n      Lear, King of Britain.\n      King of France.\n      Duke of Burgundy.\n      Duke of Cornwall.\n      Duke of Albany.\n      Earl of Kent.\n      Earl of Gloucester.\n      Edgar, son of Gloucester.\n      Edmund, bastard son to Gloucester.\n      Curan, a courtier.\n      Old Man, tenant to Gloucester.\n      Doctor.\n      Lear's Fool.\n      Oswald, steward to Goneril.\n      A Captain under Edmund's command.\n      Gentlemen.\n      A Herald.\n      Servants to Cornwall.\n\n      Goneril, daughter to Lear.\n      Regan, daughter to Lear.\n      Cordelia, daughter to Lear.\n\n      Knights attending on Lear, Officers, Messengers, Soldiers,\n        Attendants.\n\n\n\n\n<<THIS ELECTRONIC VERSION OF THE COMPLETE WORKS OF WILLIAM\nSHAKESPEARE IS COPYRIGHT 1990-1993 BY WORLD LIBRARY, INC., AND IS\nPROVIDED BY PROJECT GUTENBERG ETEXT OF CARNEGIE MELLON UNIVERSITY\nWITH PERMISSION.  ELECTRONIC 

In [13]:

# Task 3: Embeddings and Vectors

# Next, we have to convert our corpus into a "machine readable" format 

# we're going to see the actual process of creating, and then storing, 
# these embeddings, and how we can leverage that to intelligently add context to our queries.
# OpenAI API Key

# In order to access OpenAI's APIs, we'll need to provide our OpenAI API Key!


In [14]:
import os
import openai
from openai import OpenAI
%load_ext dotenv
%dotenv

In [15]:

# Vector Database

# Let's set up our vector database to hold all our documents and their embeddings!

# While this is all baked into 1 call - we can look at some of the code that powers this process to get a better understanding:

# Let's look at our VectorDatabase().__init__():

# def __init__(self, embedding_model: EmbeddingModel = None):
#         self.vectors = defaultdict(np.array)
#         self.embedding_model = embedding_model or EmbeddingModel()

# As you can see - our vectors are merely stored as a dictionary of np.array objects.

# Secondly, our VectorDatabase() has a default EmbeddingModel() which is a wrapper for OpenAI's text-embedding-3-small model.

#     Quick Info About text-embedding-3-small:

#         It has a context window of 8191 tokens
#         It returns vectors with dimension 1536



In [16]:


# We can call the async_get_embeddings method of our EmbeddingModel() on a list of str and receive a list of float back!

# async def async_get_embeddings(self, list_of_text: List[str]) -> List[List[float]]:
#         return await aget_embeddings(
#             list_of_text=list_of_text, engine=self.embeddings_model_name
#         )

# We cast those to np.array when we build our VectorDatabase():

# async def abuild_from_list(self, list_of_text: List[str]) -> "VectorDatabase":
#         embeddings = await self.embedding_model.async_get_embeddings(list_of_text)
#         for text, embedding in zip(list_of_text, embeddings):
#             self.insert(text, np.array(embedding))
#         return self

# And that's all we need to do!


In [17]:
vector_db = VectorDatabase()
vector_db = asyncio.run(vector_db.abuild_from_list(split_documents))

In [18]:


# So, to review what we've done so far in natural language:

#     We load source documents
#     We split those source documents into smaller chunks (documents)
#     We send each of those documents to the text-embedding-3-small OpenAI API endpoint
#     We store each of the text representations with the vector representations as keys/values in a dictionary

# Semantic Similarity

# The next step is to be able to query our VectorDatabase() with a str and have it return to us vectors and text that is most relevant from our corpus.

# We're going to use the following process to achieve this in our toy example:

#     We need to embed our query with the same EmbeddingModel() as we used to construct our VectorDatabase()
#     We loop through every vector in our VectorDatabase() and use a distance measure to compare how related they are
#     We return a list of the top k closest vectors, with their text representations

# There's some very heavy optimization that can be done at each of these steps - but let's just focus on the basic pattern in this notebook.

#     We are using cosine similarity as a distance metric in this example - but there are many many distance metrics you could use - like these

#     We are using a rather inefficient way of calculating relative distance between the query vector and all other vectors - there are more advanced approaches that are much more efficient, like ANN



In [19]:
vector_db.search_by_text("Who are the main characters in King Lear story?", k=3)

[("ent the candle, and we were left darkling.\n  Lear. Are you our daughter?\n  Gon. Come, sir,\n     I would you would make use of that good wisdom\n     Whereof I know you are fraught, and put away\n     These dispositions that of late transform you\n     From what you rightly are.\n  Fool. May not an ass know when the cart draws the horse?\n     Whoop, Jug, I love thee!\n  Lear. Doth any here know me? This is not Lear.\n     Doth Lear walk thus? speak thus? Where are his eyes?\n     Either his notion weakens, his discernings \n     Are lethargied- Ha! waking? 'Tis not so!\n     Who is it that can tell me who I am?\n  Fool. Lear's shadow.\n  Lear. I would learn that; for, by the marks of sovereignty,\n     Knowledge, and reason, I should be false persuaded\n     I had daughters.\n  Fool. Which they will make an obedient father.\n  Lear. Your name, fair gentlewoman?\n  Gon. This admiration, sir, is much o' th' savour\n     Of other your new pranks. I do beseech you\n     To understand

In [20]:

# Task 4: Prompts

# In the following section, we'll be looking at the role of prompts - and how they help us to guide our application in the right direction.

# In this notebook, we're going to rely on the idea of "zero-shot in-context learning".

# This is a lot of words to say: "We will ask it to perform our desired task in the prompt, and provide no examples."
# XYZRolePrompt

# Before we do that, let's stop and think a bit about how OpenAI's chat models work.

# We know they have roles - as is indicated in the following API documentation

# There are three roles, and they function as follows (taken directly from OpenAI):

#     {"role" : "system"} : The system message helps set the behavior of the assistant. For example, you can modify the personality of the assistant or provide specific instructions about how it should behave throughout the conversation. However note that the system message is optional and the model’s behavior without a system message is likely to be similar to using a generic message such as "You are a helpful assistant."
#     {"role" : "user"} : The user messages provide requests or comments for the assistant to respond to.
#     {"role" : "assistant"} : Assistant messages store previous assistant responses, but can also be written by you to give examples of desired behavior.

# The main idea is this:

#     You start with a system message that outlines how the LLM should respond, what kind of behaviours you can expect from it, and more
#     Then, you can provide a few examples in the form of "assistant"/"user" pairs
#     Then, you prompt the model with the true "user" message.

# In this example, we'll be forgoing the 2nd step for simplicities sake.


In [21]:

# XYZRolePrompt

# Here we have our system, user, and assistant role prompts.

# Let's take a peek at what they look like:

# class BasePrompt:
#     def __init__(self, prompt):
#         """
#         Initializes the BasePrompt object with a prompt template.

#         :param prompt: A string that can contain placeholders within curly braces
#         """
#         self.prompt = prompt
#         self._pattern = re.compile(r"\{([^}]+)\}")

#     def format_prompt(self, **kwargs):
#         """
#         Formats the prompt string using the keyword arguments provided.

#         :param kwargs: The values to substitute into the prompt string
#         :return: The formatted prompt string
#         """
#         matches = self._pattern.findall(self.prompt)
#         return self.prompt.format(**{match: kwargs.get(match, "") for match in matches})

#     def get_input_variables(self):
#         """
#         Gets the list of input variable names from the prompt string.

#         :return: List of input variable names
#         """
#         return self._pattern.findall(self.prompt)

# Then we have our RolePrompt which laser focuses us on the role pattern found in most API endpoints for LLMs.

# class RolePrompt(BasePrompt):
#     def __init__(self, prompt, role: str):
#         """
#         Initializes the RolePrompt object with a prompt template and a role.

#         :param prompt: A string that can contain placeholders within curly braces
#         :param role: The role for the message ('system', 'user', or 'assistant')
#         """
#         super().__init__(prompt)
#         self.role = role

#     def create_message(self, **kwargs):
#         """
#         Creates a message dictionary with a role and a formatted message.

#         :param kwargs: The values to substitute into the prompt string
#         :return: Dictionary containing the role and the formatted message
#         """
#         return {"role": self.role, "content": self.format_prompt(**kwargs)}

# We'll look at how the SystemRolePrompt is constructed to get a better idea of how that extension works:

# class SystemRolePrompt(RolePrompt):
#     def __init__(self, prompt: str):
#         super().__init__(prompt, "system")

# That pattern is repeated for our UserRolePrompt and our AssistantRolePrompt as well.
# ChatOpenAI

# Next we have our model, which is converted to a format analagous to libraries like LangChain and LlamaIndex.

# Let's take a peek at how that is constructed:

# class ChatOpenAI:
#     def __init__(self, model_name: str = "gpt-4o-mini"):
#         self.model_name = model_name
#         self.openai_api_key = os.getenv("OPENAI_API_KEY")
#         if self.openai_api_key is None:
#             raise ValueError("OPENAI_API_KEY is not set")

#     def run(self, messages, text_only: bool = True):
#         if not isinstance(messages, list):
#             raise ValueError("messages must be a list")

#         openai.api_key = self.openai_api_key
#         response = openai.ChatCompletion.create(
#             model=self.model_name, messages=messages
#         )

#         if text_only:
#             return response.choices[0].message.content

#         return response



In [22]:
# from openai_utils.prompts import (
#     UserRolePrompt,
#     SystemRolePrompt,
#     AssistantRolePrompt,
# )
# from openai_utils.chatmodel import ChatOpenAI

# chat_openai = ChatOpenAI()
# user_prompt_template = "{content}"
# user_role_prompt = UserRolePrompt(user_prompt_template)
# system_prompt_template = (
#     "You are an expert in {expertise}, you always answer in a kind way."
# )
# system_role_prompt = SystemRolePrompt(system_prompt_template)

# messages = [
#     system_role_prompt.create_message(expertise="Python"),
#     user_role_prompt.create_message(
#         content="What is the best way to write a loop?"
#     ),
# ]

# response = chat_openai.run(messages)


In [24]:

# Task 5: Retrieval Augmented Generation

# Now we can create a RAG prompt - which will help our system behave in a way that makes sense!

# There is much you could do here, many tweaks and improvements to be made!


In [25]:
RAG_PROMPT_TEMPLATE = """ \
Use the provided context to answer the user's query.

You may not answer the user's query unless there is specific context in the following text.

If you do not know the answer, or cannot answer, please respond with "I don't know".
"""

rag_prompt = SystemRolePrompt(RAG_PROMPT_TEMPLATE)

USER_PROMPT_TEMPLATE = """ \
Context:
{context}

User Query:
{user_query}
"""


user_prompt = UserRolePrompt(USER_PROMPT_TEMPLATE)

class RetrievalAugmentedQAPipeline:
    def __init__(self, llm: ChatOpenAI(), vector_db_retriever: VectorDatabase) -> None:
        self.llm = llm
        self.vector_db_retriever = vector_db_retriever

    def run_pipeline(self, user_query: str) -> str:
        context_list = self.vector_db_retriever.search_by_text(user_query, k=4)

        context_prompt = ""
        for context in context_list:
            context_prompt += context[0] + "\n"

        formatted_system_prompt = rag_prompt.create_message()

        formatted_user_prompt = user_prompt.create_message(user_query=user_query, context=context_prompt)

        return {"response" : self.llm.run([formatted_system_prompt, formatted_user_prompt]), "context" : context_list}


In [26]:
retrieval_augmented_qa_pipeline = RetrievalAugmentedQAPipeline(
    vector_db_retriever=vector_db,
    llm=chat_openai
)


In [27]:
retrieval_augmented_qa_pipeline.run_pipeline("Who are the main characters in King Lear story? ")


{'response': "I don't know.",
 'context': [("ere was good sport at his making, and the\n     whoreson must be acknowledged.- Do you know this noble\ngentleman,\n     Edmund?\n  Edm. [comes forward] No, my lord.\n  Glou. My Lord of Kent. Remember him hereafter as my honourable\n     friend.\n  Edm. My services to your lordship.\n  Kent. I must love you, and sue to know you better.\n  Edm. Sir, I shall study deserving.\n  Glou. He hath been out nine years, and away he shall again.\n                                                 Sound a sennet.\n     The King is coming.\n\n      Enter one bearing a coronet; then Lear; then the Dukes of\n      Albany and Cornwall; next, Goneril, Regan, Cordelia, with\n                              Followers.\n\n  Lear. Attend the lords of France and Burgundy, Gloucester.\n  Glou. I shall, my liege.\n                                 Exeunt [Gloucester and Edmund].\n  Lear. Meantime we shall express our darker purpose.\n     Give me the map there. Know we 

In [28]:
#Enhance your RAG application in some way - may be lets allow it to accept PDF files

In [30]:
# pip install PyPDF2

In [ ]:
import PyPDF2

def extract_text_from_pdf(pdf_path):
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        text = ""
        for page_num in range(len(reader.pages)):
            page = reader.pages[page_num]
            text += page.extract_text()
        return text

pdf_file = "data/"
extracted_text = extract_text_from_pdf(pdf_file)
print(extracted_text)

with open("data/", 'w') as file:
    file.write(extracted_text)


In [ ]:
text_loader = TextFileLoader("data/")
documents_pdf = text_loader.load_documents()
len(documents_pdf)

In [ ]:
text_splitter = CharacterTextSplitter()
split_extracted_text = text_splitter.split_texts(documents_pdf)
len(split_extracted_text)

In [ ]:
vector_db_pdf = VectorDatabase()
vector_db_pdf = asyncio.run(vector_db.abuild_from_list(split_extracted_text))

In [ ]:
vector_db_pdf.search_by_text("A deep reinforced model for abstractive summarization", k=3)

In [ ]:
RAG_PROMPT_TEMPLATE = """ \
Use the provided context to answer the user's query.

You may not answer the user's query unless there is specific context in the following text.

If you do not know the answer, or cannot answer, please respond with "I don't know".
"""

rag_prompt = SystemRolePrompt(RAG_PROMPT_TEMPLATE)

USER_PROMPT_TEMPLATE = """ \
Context:
{context}

User Query:
{user_query}
"""

user_prompt = UserRolePrompt(USER_PROMPT_TEMPLATE)

class RetrievalAugmentedQAPipeline:
    def __init__(self, llm: ChatOpenAI(), vector_db_retriever: VectorDatabase) -> None:
        self.llm = llm
        self.vector_db_retriever = vector_db_retriever

    def run_pipeline(self, user_query: str) -> str:
        context_list = self.vector_db_retriever.search_by_text(user_query, k=4)

        context_prompt = ""
        for context in context_list:
            context_prompt += context[0] + "\n"

        formatted_system_prompt = rag_prompt.create_message()

        formatted_user_prompt = user_prompt.create_message(user_query=user_query, context=context_prompt)

        return {"response" : self.llm.run([formatted_system_prompt, formatted_user_prompt]), "context" : context_list}


In [ ]:
from aimakerspace.openai_utils.prompts import (UserRolePrompt, SystemRolePrompt, AssistantRolePrompt,)
from aimakerspace.openai_utils.chatmodel import ChatOpenAI

chat_openai = ChatOpenAI()

retrieval_augmented_qa_pipeline = RetrievalAugmentedQAPipeline(vector_db_retriever=vector_db_pdf, llm=chat_openai)



In [ ]:
from pprint import pprint

questions = []
questions.append("Does a 'deep reinforced model for abstractive summarization' appear in the document?")
questions.append("Is Richard Socher one of the authors?")
questions.append("What is an encoder and decoder stack?")
questions.append("Does a 'deep reinforced model for abstractive summarization' appear in the document?")

for question in questions:
    print(retrieval_augmented_qa_pipeline.run_pipeline(question))
    #pprint(retrieval_augmented_qa_pipeline.run_pipeline(question))

